In [ ]:
# TODO
# GPU + docker

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir tb_logs/my_model/version_97

In [1]:
import import_ipynb
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import numpy as np
import pytorch_lightning as pl
import torch.nn.functional as F
from torch.optim import Adam, SGD

from sampler import *
from data_modules import *
from callbacks import *

In [2]:
def generate_data(n, output_format):
    local = sample_no_signalling(n, True, output_format)
    non_local = sample_no_signalling(n, False, output_format)
    data = local + non_local
    targets = [0] * n + [1] * n
    return data, targets

In [ ]:
data, targets = generate_data(1000, 2)

In [3]:
class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels = 1,out_channels = 32,kernel_size = (3,3))
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.conv3 = nn.Conv2d(32, 32, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(614656, 100)
        self.fc2 = nn.Linear(100, 1)
        self.sigmoid = nn.Sigmoid()
    
    def features(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        #x = self.pool(F.relu(self.conv3(x)))
        #x = self.pool(F.relu(self.conv3(x)))
        #x = self.pool(F.relu(self.conv3(x)))
        return x

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = self.fc2(x)
        return self.sigmoid(x)
    
class MLP(nn.Module):
    def __init__(self, input_len, hidden_len):
        super().__init__()
        self.fc1 = nn.Linear(input_len, hidden_len)
        self.fc2 = nn.Linear(hidden_len, hidden_len)
        self.fc3 = nn.Linear(hidden_len, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x = x.float()
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return self.sigmoid(x)

In [4]:
from torchmetrics import Accuracy
from torch.nn import BCEWithLogitsLoss

class Classifier(pl.LightningModule):
    def __init__(self, model):
        super().__init__()
        self.model = model
        self.loss = torch.nn.BCELoss()
        self.acc = Accuracy()
        
    def forward(self, x):
        return self.model(x)
    
    def configure_optimizers(self):
        return SGD(self.parameters(), lr=1e-3)
        
    def training_step(self, train_batch, batch_idx):
        x, y = train_batch
        y_hat = self.model(x)
        y = y.float().unsqueeze(1)
        loss = self.loss(y_hat, y)
        self.log('train_loss', loss)
        self.log('train acc', self.acc(torch.round(y_hat), y.int()))
        return loss

    def validation_step(self, val_batch, batch_idx):
        x, y = val_batch
        y_hat = self.model(x)
        y = y.float().unsqueeze(1)
        val_loss = self.loss(y_hat, y)
        self.log('val loss', val_loss)
        self.log('val acc', self.acc(torch.round(y_hat), y.int()))

        return val_loss
    
    def test_step(self, batch, batch_idx):
        x, y = batch

        y_hat = self.model(x)
        y = y.float().unsqueeze(1)
        loss = self.loss(y_hat, y)
        output = dict({
                'test_loss': loss,
            'test_acc': self.acc(torch.round(y_hat), y.int()),        })
        return output

In [ ]:
from pl_bolts.callbacks import PrintTableMetricsCallback
from pytorch_lightning.loggers import WandbLogger 
from pytorch_lightning.loggers import TensorBoardLogger
from data_modules import MyDatasetOfImages
logger = TensorBoardLogger("tb_logs", name="my_model")
transform = transforms.Compose([
    transforms.ToTensor(),
#    transforms.Normalize([0,0,0], [1,1,1])
#    transforms.Resize(100)
])
dm = MyDataModule(data, targets, transform, 16, MyDatasetOfImages)
clf = Classifier(ConvNet())
trainer = pl.Trainer(max_epochs=500,
                     gpus=1, callbacks=[PrintTableMetricsCallback2(20)],
                     logger=logger)
trainer.fit(clf, dm)

In [5]:
data2, targets2 = generate_data(1000, 1)

In [7]:
from pl_bolts.callbacks import PrintTableMetricsCallback
from pytorch_lightning.loggers import WandbLogger 
from pytorch_lightning.loggers import TensorBoardLogger
from data_modules import MyDatasetOfImages
logger = TensorBoardLogger("tb_logs", name="my_model")
transform = transforms.Compose([
    transforms.ToTensor(),
#    transforms.Normalize([0,0,0], [1,1,1])
#    transforms.Resize(100)
])

dm2 = MyDataModule(data2, targets2, transform, 16, MyDatasetOfImages)
clf2 = Classifier(ConvNet())
trainer2 = pl.Trainer(max_epochs=500, 
                     callbacks=[PrintTableMetricsCallback2(20)],
                     logger=logger)
trainer2.fit(clf2, dm2)

GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/home/johnny/.virtualenvs/jup/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:1579: UserWarning: GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`.
  rank_zero_warn(

  | Name  | Type     | Params
-----------------------------------
0 | model | ConvNet  | 61.5 M
1 | loss  | BCELoss  | 0     
2 | acc   | Accuracy | 0     
-----------------------------------
61.5 M    Trainable params
0         Non-trainable params
61.5 M    Total params
245.975   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/home/johnny/.virtualenvs/jup/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:407: UserWarning: The number of training samples (23) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

/home/johnny/.virtualenvs/jup/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:685: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [ ]:
from pl_bolts.callbacks import PrintTableMetricsCallback
from pytorch_lightning.loggers import WandbLogger 
from pytorch_lightning.loggers import TensorBoardLogger

logger = TensorBoardLogger("tb_logs", name="my_model")
transform = transforms.Compose([
 #   transforms.ToTensor(),
#    transforms.Resize(100)
])
dm = MyDataModule(data, targets, transform, 16)
clf = Classifier(MLP(16, 100))
trainer = pl.Trainer(max_epochs=500,
                     gpus=1, callbacks=[PrintTableMetricsCallback2(20)],
                     logger=logger)
trainer.fit(clf, dm)

In [ ]:
next(iter(dm.train_dataset))

In [ ]:
data[2]

In [ ]:
dm.setup(None)

In [ ]:
dm = MyDataModule(data, targets, None, 16)

In [ ]:
from pl_bolts.callbacks import PrintTableMetricsCallback
from pytorch_lightning.loggers import WandbLogger 
from pytorch_lightning.loggers import TensorBoardLogger

logger = TensorBoardLogger("tb_logs", name="my_model")
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize(100)
])
dm = MNISTDataModule()
clf2 = Classifier() 
trainer = pl.Trainer(max_epochs=25,
                     gpus=1, callbacks=[PrintTableMetricsCallback2(20)],
                     logger=logger)
trainer.fit(clf2, dm)

In [ ]:
a = clf.model.features(next(iter(dm.train_dataloader()))[0])

In [ ]:
plt.figure(figsize=(8, 8))
for i in range(16):
    ax = plt.subplot(4,4,i+1)
    ax.imshow(a[0,i,:,:].detach().numpy())

In [ ]:
features = clf2.model.features(next(iter(dm.train_dataloader()))[0])

In [ ]:
plt.figure(figsize=(8, 8))
for i in range(16):
    ax = plt.subplot(4,4,i+1)
    ax.imshow(features[2,i,:,:].detach().numpy(), cmap='gray')